In [260]:
import os
from contextlib import ExitStack

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
import math

import torch
import torchtext
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchmetrics.regression import MeanSquaredError
from torchmetrics.regression import R2Score

In [261]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#get device that we are using
#torch.cuda.device_count()
#torch.cuda.current_device()
#torch.cuda.get_device_name(0)

Using cuda device


In [262]:
#data processing
with ExitStack() as stack:
    data = sio.loadmat('Random.mat')

    # Extract the data
    C0 = np.array(data['C0'])
    Seq = np.array(data['Seq'])
    SS = np.array(data['SS'])
C0[0][0]
#data = np.concatenate((Seq, C0))

-0.081574

In [263]:
#split data into test train

x_train, x_test, y_train, y_test = train_test_split(
    Seq, C0, test_size=0.1, random_state=42
)

In [264]:
#adapted from https://towardsdatascience.com/custom-datasets-in-pytorch-part-2-text-machine-translation-71c41a3e994e

class Vocabulary:
    '''
    __init__ is called when object is initiated, to initiate vocab dictionaries
    '''
    def __init__(self):
        #initiate index to token dict
        # PAD: padding to match sentence lengths
        # SOS: start token
        # EOS: end token
        # UNK: words not found in the vocab 
        self.itos = {0: '<PAD>', 1:'<SOS>', 2:'<EOS>', 3:'<UNK>'}
        #initiate token to index dict
        self.stoi = {k:j for j,k in self.itos.items()} 
    
    '''
    __len__ used by dataloader to create batches (maybe need?)
    '''
    def __len__(self):
        return len(self.itos)
    
    '''tokenizer converts each character in RNA sequence to an element in a list'''
    @staticmethod
    def tokenizer(text):
        return [tok.lower() for tok in text]
    
    '''build the vocab: map index to string, string to index'''
    def build_vocab(self):
        idx = 4 #start at 4 b/c 0 to 3 are taken
        residues = ["a", "t", "c", "g"]
        for res in residues:
            self.stoi[res] = idx
            self.itos[idx] = res
            idx+=1

    def numericalize(self, text):
        tokenized_text = self.tokenizer(text)
        numericalized_text = []
        for token in tokenized_text:
            if token in self.stoi.keys():
                numericalized_text.append(self.stoi[token])
            else:
                numericalized_text.append(self.stoi['<UNK>'])
        return numericalized_text

In [265]:
#####################
#Build Train Dataset#
#####################

class Train_Dataset(Dataset):
    '''
    Variables
    x_train: training sequences
    y_train: training flexibility index
    transform: if we want to add augmentation
    '''

    def __init__(self, x_train, y_train, transform=None):
        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

        #create vocabulary (only for x data):
        self.source_vocab = Vocabulary()
        self.source_vocab.build_vocab()
    
    def __len__(self):
        return len(self.x_train)
    
    '''__getitem___ runs on 1 item at a time'''

    def __getitem__(self, index):
        source_text = self.x_train[index]
        if self.transform is not None:
            source_text = self.transform(source_text)
        
        #numericalize texts
        numericalized_source = [self.source_vocab.stoi["<SOS>"]]
        numericalized_source += self.source_vocab.numericalize(source_text)
        numericalized_source.append(self.source_vocab.stoi["<EOS>"])

        target = self.y_train[index][0]

        return torch.tensor(numericalized_source), torch.tensor(target, dtype=torch.float)

In [266]:
class Validation_Dataset(Dataset):
    '''
    Variables
    x_train: training sequences
    y_train: training flexibility index
    transform: if we want to add augmentation
    '''

    def __init__(self, train_dataset, x_test, y_test, transform=None):
        self.train_dataset = train_dataset
        self.x_test = x_test
        self.y_test = y_test
        self.transform = transform
    
    def __len__(self):
        return len(self.x_test)
    
    '''__getitem___ runs on 1 item at a time'''

    def __getitem__(self, index):
        source_text = self.x_test[index]
        if self.transform is not None:
            source_text = self.transform(source_text)
        
        #numericalize texts
        numericalized_source = [self.train_dataset.source_vocab.stoi["<SOS>"]]
        numericalized_source += self.train_dataset.source_vocab.numericalize(source_text)
        numericalized_source.append(self.train_dataset.source_vocab.stoi["<EOS>"])

        target = self.y_test[index][0]

        return torch.tensor(numericalized_source), torch.tensor(target, dtype=torch.float)

In [267]:
#########
#Collate#
#########

'''Adds padding (shouldn't need for this)'''
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    #__call__: runs by default when MyCollate is created
    def __call__(self, batch):
        source = [item[0] for item in batch]
        source = pad_sequence(source, batch_first=False, padding_value=self.pad_idx)

In [268]:
#loader functions
def get_train_loader(dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    #get pad_idx for collate fn
    pad_idx = dataset.source_vocab.stoi['<PAD>']
    print(pad_idx)
    #define loader
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory) 
    return loader

def get_valid_loader(dataset, train_dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True):
    pad_idx = train_dataset.source_vocab.stoi['<PAD>']
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory)
    return loader

In [269]:
#transformer adapted from https://n8henrie.com/2021/08/writing-a-transformer-classifier-in-pytorch/

class PositionalEncoding(nn.Module):
    '''https://pytorch.org/tutorials/beginner/transformer_tutorial.html'''

    #vocab size is 8 i think
    
    #nvm i think vocab size needs to be larger than seq size: this way it can shrink down to the seq size
    #positional encoder encodes a unique function for each index and position in the text
    
    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0)/d_model)
        )
        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)
    
    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [270]:
class TransformerRegressionModel(nn.Module):
    '''text regression model using a transformer'''

    def __init__(
            self,
            vocab_size,
            d_model,
            n_head,
            dim_feedforward,
            num_layers,
            dropout,
            classifier_dropout,
            activation = "relu"
    ):
        
        super().__init__()
        self.model_type = "Transformer"
        self.vocab_size = vocab_size
        assert d_model % n_head == 0 #assert that the number of attention heads divides into the size of the embedding (d_model)
        
        #may need to make vocab_size a bit bigger (same as encoding) for the embedding and encoding to work
        self.emb = nn.Embedding(5000, d_model) #embedding of size d_model
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.regressor = nn.Linear(d_model, 1)
        self.d_model = d_model
    
    def forward(self, x):
        x=self.emb(x) * math.sqrt(self.d_model)
        x=self.pos_encoder(x)
        x=self.transformer_encoder(x)
        x=x.mean(dim=1) #taking the mean along the sequence size dimension is a common way to process transformer output
        x=self.regressor(x)
        x=x.squeeze(-1)
        return x

In [271]:
train_data = Train_Dataset(x_train, y_train)
print(x_train[0], y_train[0])
train_data[0]

CAGCCTTCTTACTCTTACGCCCAGGGCATACGGGGGGTTCCCGATCGATG [0.27438]


(tensor([1, 6, 4, 7, 6, 6, 5, 5, 6, 5, 5, 4, 6, 5, 6, 5, 5, 4, 6, 7, 6, 6, 6, 4,
         7, 7, 7, 6, 4, 5, 4, 6, 7, 7, 7, 7, 7, 7, 5, 5, 6, 6, 6, 7, 4, 5, 6, 7,
         4, 5, 7, 2]),
 tensor(0.2744))

In [272]:
test_data = Validation_Dataset(train_data, x_test, y_test)
print(x_test[0], y_test[0])
test_data[0]

CGCGGCTGGAAGTAACCTACTCAAACATTGTGCATCCGCCTGGCTCGGGT [-0.56108]


(tensor([1, 6, 7, 6, 7, 7, 6, 5, 7, 7, 4, 4, 7, 5, 4, 4, 6, 6, 5, 4, 6, 5, 6, 4,
         4, 4, 6, 4, 5, 5, 7, 5, 7, 6, 4, 5, 6, 6, 7, 6, 6, 5, 7, 7, 6, 5, 6, 7,
         7, 7, 5, 2]),
 tensor(-0.5611))

In [273]:
train_loader = get_train_loader(train_data, 32)
source = next(iter(train_loader))[0]
flex = next(iter(train_loader))[1]

0


In [274]:
test_loader = get_valid_loader(test_data, train_data, 32)

In [280]:
vocab_size = train_data.source_vocab.__len__()
d_model = 200
n_head = 5
dim_feedforward = 50
num_layers = 6
dropout = 0.1
classifier_dropout = 0.1

#create the model and then move it to the GPU
model = TransformerRegressionModel(vocab_size, d_model, n_head, dim_feedforward, num_layers, dropout, classifier_dropout).to(device)

In [283]:
import time
import copy

criterion = nn.MSELoss()
lr = 0.01
optimizer = torch.optim.SGD((p for p in model.parameters() if p.requires_grad), lr)

torch.manual_seed(0)

def train(model: nn.Module):
    model.train()
    epoch_train_loss = 0

    #begin batching
    for idx, batch in enumerate(iter(train_loader)):
        predictions = model(batch[0].to(device))
        scores = batch[1].to(device)

        loss = criterion(predictions, scores)

        #can add accuracy code, e.g. threshold within 10%; MSE is also a good indicator of correctness
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        epoch_train_loss += loss.item()
        #epoch_train_loss /= len(train_data)
        
        #print(f"{predictions=} | {scores=}")

    print(f"{epoch_train_loss=}")

def evaluate(model: nn.Module):
    model.eval()
    epoch_test_loss = 0
    with torch.no_grad():
        for idx, batch in enumerate(iter(test_loader)):
            output = model(batch[0].to(device))
            scores = batch[1].to(device)

            test_loss = criterion(output, scores)
            epoch_test_loss += test_loss.item()
            #epoch_test_loss /= len(test_data)

    print(f"{epoch_test_loss=}")

In [285]:
#train the model!
epochs = 5

print("starting")
for epoch in range(1, epochs + 1):
    print(f"{epoch=}")
    epoch_start_time = time.time()
    train(model)
    evaluate(model)
    elapsed = time.time() - epoch_start_time
    print(f"{elapsed=}")

starting
epoch=1
predictions=tensor([ 0.0573, -0.0195,  0.0335,  0.0087,  0.0038, -0.0186, -0.0358,  0.0154,
        -0.0380, -0.0247, -0.0837,  0.0010, -0.0019, -0.0530, -0.0201,  0.0169,
         0.0034,  0.0385, -0.0111,  0.0345, -0.0354,  0.0122, -0.0181, -0.0399,
        -0.0229,  0.0095, -0.0028,  0.0077, -0.0192,  0.0017, -0.0058,  0.0057],
       device='cuda:0', grad_fn=<SqueezeBackward1>) | scores=tensor([-0.3046, -0.1960, -0.2345,  0.4587,  0.7501,  0.2491,  0.4303, -0.0941,
        -0.1522, -0.4486,  0.1525, -0.0298,  0.6714,  0.4239, -0.0779, -0.3844,
        -0.0897, -0.4609,  0.3305,  1.1015, -0.0511,  0.3707,  0.0311, -0.4603,
        -0.3427, -0.0965, -0.2574, -0.7717,  0.1996,  0.0429,  0.4400, -0.7527],
       device='cuda:0')
predictions=tensor([0.0534, 0.0967, 0.1021, 0.0397, 0.0994, 0.0874, 0.0847, 0.0362, 0.0516,
        0.0615, 0.0846, 0.0391, 0.0409, 0.1063, 0.0587, 0.0383, 0.0889, 0.0528,
        0.1132, 0.0265, 0.0981, 0.0827, 0.0889, 0.0361, 0.0651, 0.0785, 

In [ ]:
from torchtext.datasets import IMDB

train_iter = iter(IMDB(split='train'))
next(train_iter)

(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

In [227]:
for idx, batch in enumerate(iter(train_loader)):
    print(batch[1].size())

torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32